In [1]:
# from NLPutils.models.dilatedconv import DilatedConv
from NLPutils.models.dconv3 import DConv

In [2]:
# from NLPutils.trainers.dconv_trainer import DConvTrainer
from NLPutils.trainers.dconv2 import DConvTrainer

In [3]:
train = '/data/ner/conll/eng.train'
valid = '/data/ner/conll/eng.testa'
test = '/data/ner/conll/eng.testb'
word_embed_loc = '/data/embeddings/GoogleNews-vectors-negative300.bin'
# word_embed_loc = '/data/embeddings/glove.6B/glove.6B.300d.txt'
valsplit=0.15

In [4]:
from NLPutils.util.conll_util import *
from NLPutils.embeddings import Word2VecModel, RandomInitVecModel, GloVeModel

maxs, maxw, vocab_ch, vocab_word = conllBuildVocab([train, valid,
                                                    test])

# Vocab LUTs
word_vocab = None
char_vocab = None

word_vec = Word2VecModel(word_embed_loc, vocab_word, 0.25)
# word_vec = GloVeModel(word_embed_loc, vocab_word, 0.25)
word_vocab = word_vec.vocab

# if FLAGS.charsz != FLAGS.wsz and FLAGS.cbow is True:
#     print('Warning, you have opted for CBOW char embeddings, but have provided differing sizes for char embedding depth and word depth.  This is not possible, forcing char embedding depth to be word depth ' + FLAGS.wsz)
#     FLAGS.charsz = FLAGS.wsz

char_vec = RandomInitVecModel(16, vocab_ch, 0.25)
char_vocab = char_vec.vocab

f2i = {"<PAD>":0}

ts, f2i, _ = conllSentsToIndices(train, word_vocab, char_vocab, maxs, maxw, f2i, 3)
print(len(ts))
print('Loaded  training data')

if valid is not None:
    print('Using provided validation data')
    vs, f2i,_ = conllSentsToIndices(valid, word_vocab, char_vocab, maxs, maxw, f2i, 3)
else:
    ts, vs = validSplit(ts, valsplit)
    print('Created validation split')


es, f2i,txts = conllSentsToIndices(test, word_vocab, char_vocab, maxs, maxw, f2i, 3)
print('Loaded test data')


14986
Loaded  training data
Using provided validation data
Loaded test data


In [5]:
print('word embed weights shape: ', word_vec.weights.shape)
print( 'max sentence len', maxs)
print('f2i length', len(f2i))

word embed weights shape:  (30291, 300)
max sentence len 124
f2i length 9


In [7]:
import tensorflow as tf

trainer = DConvTrainer(None, DConv, 'conll-ner-dconv-strubell-5')
trainer.train('ner', ts, f2i, vs, es,
             char_vec,
             word_vec,
             'eval',
              batchsz=2**5,
              optim='adam',
              eta=0.0005,
              epochs=500,
              dropout=0.15,
              patience=50,
              cfiltsz='1,3,5,7',
             maxlen=maxs,
             maxw=maxw,
              num_filt=300,
             num_layers=3,
             num_iterations=3,
             fscore=1,
              viz=1,
             crf=True)

#1 -- 86.55 dropout at .45 and 300 filters. stopped at epoc 24.
#2 -- 86.7 dropout at .35, 350 filters.
#3 -- added dilation 1 conv at end of block. 86.28
#4 -- 3 layers, 3 iters. got much worse. peeked at 91.8 in training, started dropping.
#5 -- layers in block should be 1,2,1 from strubell's code. lowered dropout to .15 -- layers would have been 1,1,1.
#6 -- clipped gradients'
#7 -- 3 layers, 2 iterations.

#strubell-1 -- just trying it. only hits 85.5
#strubell-2 -- glove embeddings -- holy shit. only hits 86 IN VALIDATION
#strubell-3 -- glove embeddings hwy embeddings. hits almost 86.7 soo a little better?
#strubell-4 -- trying word2vec again. 86.3
#strubell-5 -- num_iter to 3

input_feats Tensor("ner/ExpandDims:0", shape=(?, 1, 124, 420), dtype=float32)
filter_shape [1, 3, Dimension(420), 300]
h0 Tensor("ner/relu:0", shape=(?, 1, 124, 300), dtype=float32)
last_output Tensor("ner/concat_1:0", shape=(?, 1, 124, 300), dtype=float32)
block output Tensor("ner/block/iterated-block/relu_2:0", shape=(?, ?, ?, 300), dtype=float32)
h_concat_squeeze Tensor("ner/block/Squeeze:0", shape=(?, ?, 300), dtype=float32)
h_concat_flat Tensor("ner/block/Reshape:0", shape=(?, 300), dtype=float32)
input_to_pred Tensor("ner/block/hidden_dropout/dropout/mul:0", shape=(?, 300), dtype=float32)
proj_width 300
scores Tensor("ner/block/output/scores:0", shape=(?, 9), dtype=float32)
unflat_scores Tensor("ner/block/output/Reshape:0", shape=(?, 124, 9), dtype=float32)
block output Tensor("ner/block_1/iterated-block/relu_2:0", shape=(?, ?, ?, 300), dtype=float32)
h_concat_squeeze Tensor("ner/block_1/Squeeze:0", shape=(?, ?, 300), dtype=float32)
h_concat_flat Tensor("ner/block_1/Reshape:0", s

/opt/conda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Setting up word embedding visualization
Writing metadata
Training epoch 1.
Train (Loss 0.0780) (185.126 sec)
Validation (F1 = 0.8433) (Acc 50067/51409 = 0.9739) (9.690 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 2.
	(last improvement @ 1)
Train (Loss 0.0275) (184.544 sec)
Validation (F1 = 0.8854) (Acc 50366/51409 = 0.9797) (9.479 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 3.
	(last improvement @ 2)
Train (Loss 0.0170) (184.135 sec)
Validation (F1 = 0.8940) (Acc 50455/51409 = 0.9814) (9.470 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 4.
	(last improvement @ 3)
Train (Loss 0.0119) (184.240 sec)
Validation (F1 = 0.8693) (Acc 50256/51409 = 0.9776) (9.480 sec)
Training epoch 5.
	(last improvement @ 3)
Train (Loss 0.0081) (184.122 sec)
Validation (F1 = 0.8978) (Acc 50488/51409 = 0.9821) (9.470 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 6.
	(last improvement @ 5)
Train (Loss 0.0059) (184.112 sec)
Validation 

KeyboardInterrupt: 